# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2
# Modified by Julián Ferreiro (NetID: jif245)

In [1]:
SQL_SOURCE = 'https://jif245.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()


SyntaxError: invalid syntax (<ipython-input-1-de7489eafe75>, line 23)

In [ ]:
test = '''
SELECT *
FROM citibike
Limit 5
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

### Task 1

### Sort data by start_station_id, tripduration
- Only checking trips with duration <= 3 hours
• Only show the top/last 10 records (aka head and tail in SQL) 

In [ ]:
Task1a = '''
SELECT * FROM citibike
Where tripduration<3*60*60
Order by start_station_id desc, tripduration desc
Limit 10
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task1a)), sep=',')

### List all unique start_station_id values 

In [ ]:
Task1b = '''
SELECT DISTINCT start_station_id FROM citibike
Limit 5
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task1b)), sep=',')

### Count the number of trips (aka wc -l in SQL) 

In [ ]:
Task1c = '''
SELECT  count (cartodb_id)
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task1c)), sep=',')

### Find the average/min/max trip duration

In [ ]:
Task1d = '''
SELECT AVG(tripduration), Min(tripduration),max(tripduration) From citibike

'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task1d)), sep=',')

### Task 2 — Working with date/time

### Selecting trips started on Feb-02-2015 only

In [ ]:
Task2a = '''
select * from citibike
where starttime>='2015-02-02 00:00'
and starttime<'2015-02-03 00:00'
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task2a)), sep=',')

### Selecting trips started on the weekends

In [2]:
Task2b = '''
select * from citibike
where extract (dow from starttime) in (0,6)
'''

In [3]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task2b)), sep=',')

NameError: name 'pd' is not defined

#### What are average trip duration during weekends?

In [ ]:
Task2c = '''
select avg(tripduration)
from citibike
where extract (dow from starttime) in (0,6)
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task2c)), sep=',')

#### Can we do the same for weekday?

In [ ]:
Task2d = '''
select avg(tripduration)
from citibike
where extract (dow from starttime) in (1,2,3,4,5)
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task2d)), sep=',')

### Task 3 — Working with Space

#### Showing the list of start station locations

In [ ]:
Task3a = '''
select start_station_id, (start_station_latitude, start_station_longitude)
from citibike
Limit 5
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task3a)), sep=',')

#### Using GROUP BY
#### Showing the number of trips started per station

In [ ]:
Task3b = '''
select start_station_id,count (cartodb_id)
from citibike
group by start_station_id
limit 5
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task3b)), sep=',')

#### only for stations within 500m of Time Square!

In [ ]:
Task3c = '''
select start_station_id, count(tripduration) as numtrips
    
from citibike

where st_dwithin(cdb_latlng(start_station_latitude,start_station_longitude)::geography,
                  cdb_latlng(40.757,-73.9857)::geography,500)
group by start_station_id

'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task3c)), sep=',')

### Task 4 — Putting it all together
• Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare!

In [ ]:
Task4a = '''

select start_station_id, avg(tripduration) as max_avg
    
from citibike
where extract (dow from starttime) in (0,6) and st_dwithin(cdb_latlng(start_station_latitude,start_station_longitude)::geography,
                  cdb_latlng(40.757,-73.9857)::geography,500)
group by start_station_id
Order by max_avg desc
Limit 1

'''


In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task4a)), sep=',')

Extra: create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines. 

In [ ]:
Task4b = '''
select st_makeline(cdb_transformtowebmercator(cdb_latlng(start_station_latitude,start_station_longitude)),cdb_transformtowebmercator(cdb_latlng(end_station_latitude,end_station_longitude))) as the_geom_webmercator,
    min(cartodb_id) as cartodb_id,
    count(tripduration) as numtrips

from citibike

where st_dwithin(cdb_latlng(start_station_latitude,start_station_longitude)::geography,
                  cdb_latlng(40.757,-73.9857)::geography,
                  500)
                 and tripduration<7200
group by start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude

'''


In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(Task4b)), sep=',')

In [ ]:
#This last point needs to be visualized in Carto